<a href="https://colab.research.google.com/github/svetoslavseo/google-trends-automation/blob/main/Google_Trends_%2B_Knowledge_panel_API_%2B_DataForSEO_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feedparser
import feedparser
import pandas as pd
import requests
from datetime import datetime
import pytz
import os
from xml.etree import ElementTree as ET
import openai

def fetch_google_trends_rss():
    # URL for Google Trends RSS feed
    google_trends_rss_url = "https://trends.google.com/trending/rss?geo=GB"

    # Fetch the RSS feed as raw XML
    response = requests.get(google_trends_rss_url)
    if response.status_code != 200:
        print("Failed to fetch the feed.")
        return None
    raw_feed = response.content

    # Parse the RSS feed
    feed = feedparser.parse(raw_feed)

    # Check if the feed was parsed successfully
    if not feed.entries:
        print("Failed to fetch the feed or no entries available.")
        return None

    # Parse the raw XML for extracting <ht:news_item_title>
    root = ET.fromstring(raw_feed)

    # Extract the data into a list of dictionaries
    trends_data = []
    max_titles_count = 0  # Track the maximum number of news titles per item for dynamic column creation
    for entry in feed.entries:
        # Convert published time to UK time
        utc_time = datetime.strptime(entry.published, "%a, %d %b %Y %H:%M:%S %z")
        uk_time = utc_time.astimezone(pytz.timezone("Europe/London"))

        # Extract news titles under <ht:news_item> for the current <item>
        news_titles = []
        for item in root.findall(".//item"):
            item_title = item.find("title").text
            if item_title == entry.title:
                for news_item in item.findall(".//{https://trends.google.com/trending/rss}news_item"):
                    news_title = news_item.find("{https://trends.google.com/trending/rss}news_item_title")
                    if news_title is not None:
                        news_titles.append(news_title.text)
                break

        max_titles_count = max(max_titles_count, len(news_titles))  # Update maximum title count

        # Base trend information
        trend_info = {
            "title": entry.title,
            "published": uk_time.strftime("%Y-%m-%d %H:%M:%S %Z"),
            "summary": entry.summary,
            "link": entry.link,
            "approx_traffic": getattr(entry, 'ht_approx_traffic', 'N/A'),
            "picture_url": getattr(entry, 'ht_picture', 'N/A'),
            "picture_source": getattr(entry, 'ht_picture_source', 'N/A')
        }

        # Add dynamic columns for news titles
        for i, news_title in enumerate(news_titles):
            trend_info[f"news_title_{i + 1}"] = news_title

        trends_data.append(trend_info)

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(trends_data)

    # Fill missing dynamic columns with None if some rows have fewer titles
    for i in range(1, max_titles_count + 1):
        if f"news_title_{i}" not in df.columns:
            df[f"news_title_{i}"] = None

    return df

def categorize_with_knowledge_graph(keyword):
    api_key = "<YOUR-GOOGLE-KNOWLEDGE-GRAPH-API-KEY"  # Replace with your Google Knowledge Graph API key
    url = f"https://kgsearch.googleapis.com/v1/entities:search?query={keyword}&key={api_key}&limit=1"
    response = requests.get(url).json()

    if 'itemListElement' in response and response['itemListElement']:
        entity = response['itemListElement'][0].get('result', {})
        return entity.get('description', 'Unknown'), entity.get('@type', ['Unknown'])
    return 'Unknown', ['Unknown']

def check_serp_for_top_stories_and_telegraph(keyword):
    # Replace with your DataForSEO API credentials
    username = "<USER-NAME"
    password = "<PASSWORD"
    url = "https://api.dataforseo.com/v3/serp/google/organic/live/advanced"

    payload = {
        "keyword": keyword,
        "location_code": 2840,  # United Kingdom location code
        "language_code": "en",
        "device": "mobile"
    }

    response = requests.post(url, auth=(username, password), json=[payload])
    response_json = response.json()

    # Check for errors
    if 'tasks' in response_json and response_json['tasks']:
        task = response_json['tasks'][0]
        if task['result'] and task['result'][0]['items']:  # Check for SERP results
            serp_items = task['result'][0]['items']
            top_stories_found = "False"
            telegraph_in_top_stories = "False"
            telegraph_in_organic = "False"

            for item in serp_items:
                if item.get('type') == 'top_stories':
                    top_stories_found = "True"
                    for story in item.get('items', []):
                        if 'www.telegraph.co.uk' in story.get('url', ''):
                            telegraph_in_top_stories = "True"
                            break

                elif item.get('type') == 'organic':  # Check organic results
                    if 'www.telegraph.co.uk' in item.get('url', ''):
                        telegraph_in_organic = "True"

            return top_stories_found, telegraph_in_top_stories, telegraph_in_organic
    return "False", "False", "False"

def main():
    # Fetch Google Trends data
    df = fetch_google_trends_rss()

    if df is not None:
        output_file = "google_trends_with_categories.csv"

        # Check if file exists
        if os.path.exists(output_file):
            # Load existing data
            existing_df = pd.read_csv(output_file)
            # Append new data
            df = pd.concat([existing_df, df], ignore_index=True)

        # Remove duplicate titles, keeping the first occurrence
        df = df.drop_duplicates(subset=['title'], keep='first')

        # Add categories using Google Knowledge Graph API
        print("Fetching categories using Google Knowledge Graph API...")
        df[['Category', 'EntityType']] = df['title'].apply(
            lambda x: pd.Series(categorize_with_knowledge_graph(x))
        )

        # Check SERP for Top Stories and Telegraph
        print("Checking SERP for Top Stories and Telegraph...")
        df[['Top_Stories', 'Telegraph_in_Top_Stories','Telegraph_in_Organic_Results']] = df['title'].apply(
            lambda x: pd.Series(check_serp_for_top_stories_and_telegraph(x))
        )

        # Save the cleaned and updated DataFrame back to the file
        df.to_csv(output_file, index=False)

        # Display the DataFrame in a table format
        print("\nGoogle Trends Data with Summaries:\n")
        print(df.to_markdown(index=False))

        print(f"\nData has been saved to '{output_file}'.")

if __name__ == "__main__":
    main()

df_2 = pd.read_csv("google_trends_with_categories.csv")
# Remove the '+' character from the approx_traffic column
df_2['approx_traffic'] = df_2['approx_traffic'].str.replace('+', '', regex=False)

# Drop the 'summary' column
df_2 = df_2.drop(columns=['summary'])

# Optionally, save the cleaned data back to a CSV
df_2.to_csv('cleaned_file.csv', index=False)

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import LdaModel
from gensim.corpora import Dictionary



# Create DataFrame
df_3 = df_2 = pd.read_csv("cleaned_file.csv")

# Combine news titles into a single text column for topic modeling
df_3["combined_news_titles"] = df_3["news_title_1"] + " " + df_3["news_title_2"] + " " + df_3["news_title_3"]

# Tokenize the text
tokenized_texts = [doc.split() for doc in df_3["combined_news_titles"]]

# Create dictionary and corpus for LDA
dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Train LDA model with 4 topics
lda_model = LdaModel(corpus, num_topics=4, id2word=dictionary, passes=10)

# Get the topic distribution for each document
df_3["LDA_Topic"] = [
    max(lda_model.get_document_topics(dictionary.doc2bow(doc)), key=lambda x: x[1])[0]
    for doc in tokenized_texts
]

# Display topics
topics = lda_model.print_topics(num_words=5)

# Print the topics
for topic in topics:
    print(topic)

# Show the results
df_3.to_csv("lda_categorized_news.csv", index=False)  # Save results to CSV


#Topic 0: Likely related to travel, with keywords like Top, 5, beach, destinations.
#Topic 1: Focuses on sports, with keywords like wins, Champions, title, Manchester, final.
#Topic 2: Seems related to economy/finance, including words like Stock, markets, projected.
#Topic 3: Strongly connected to technology, featuring AI, new, Tech, companies, revolutionizing.


import os
import pandas as pd
from openai import OpenAI
from google.colab import auth

# Authenticate and set up OpenAI API key
auth.authenticate_user()
os.environ["OPENAI_API_KEY"] = "<REPLACE-WITH-YOUR-OPENAI-API-KEY"  # Set your OpenAI API key

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # Ensure API key is set
)

# Load the dataset
df_4 = pd.read_csv("lda_categorized_news.csv")

# Ensure the required column exists
df_4["combined_news_titles"] = df_4.get("news_title_1", "") + " " + df_4.get("news_title_2", "") + " " + df_4.get("news_title_3", "") + df_4.get("Category", "")

# Function to classify news using OpenAI API
def classify_news_department(news_titles):
    if not client.api_key:
        raise ValueError("OpenAI API key is not set. Please set the OPENAI_API_KEY environment variable.")

    prompt = f"""Based on the following news headlines and the category from Google Knowledge Panel at the end:
    {news_titles}

    Which department does this news belong to? Choose from:
    - News (Politics, Business, Economy, General)
    - Royal Family
    - Sports
    - Entertainment
    - Technology
    - Science & Health
    - Travel
    - Lifestyle

    Return only the department name."""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a news classification expert."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

# Function to classify news using OpenAI API
def title_summary(news_titles):
    if not client.api_key:
        raise ValueError("OpenAI API key is not set. Please set the OPENAI_API_KEY environment variable.")

    prompt = f"""Based on the provided news title, write a short summary of the titles by mentioning the main topic.
    {news_titles}"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert in summarising news in a newsparer."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip()

# Apply summary to each row
df_4["News_Summary"] = df_4["combined_news_titles"].apply(lambda x: title_summary(x) if x.strip() else "Unknown")

# Apply classification to each row
df_4["News_Department"] = df_4["combined_news_titles"].apply(lambda x: classify_news_department(x) if x.strip() else "Unknown")

# Save results to Google Drive or Colab environment, keeping all original columns
df_4.to_csv("openai_categorized_news.csv", index=False)

print(df_4.to_markdown(index=False))


from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Open the Google Sheet
sh = gc.open_by_key('<GOOGLE-SPREADSHEET-KEY>')  # Google Sheet URL key
worksheet = sh.worksheet("Sheet1")

# Load existing data from the sheet into a DataFrame
existing_df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how='all')

# Load the new data
new_df = pd.read_csv("openai_categorized_news.csv")  # Read from the CSV file

# Append new data to existing data
if not existing_df.empty:
    combined_df = pd.concat([existing_df, new_df], ignore_index=True).drop_duplicates(subset=['title'], keep='first')
else:
    combined_df = new_df

# Write the updated DataFrame back to the Google Sheet
worksheet.clear()  # Clear existing data on the sheet
set_with_dataframe(worksheet, combined_df)  # Write the combined data

print("Data has been successfully appended to the Google Sheet.")



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=418597bdb61a37721ce7703a3e70dc0b71c4fe4f89d0e0a02616b595e2411a53
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
